빅분기 QnA : https://colab.research.google.com/drive/1D_wMv23fgkNpgxyoAV6r8PqUX62qybQM

lightgbm : https://for-my-wealthy-life.tistory.com/24
- https://lightgbm.readthedocs.io/en/latest/index.html

xgboost : https://hwi-doc.tistory.com/entry/%EC%9D%B4%ED%95%B4%ED%95%98%EA%B3%A0-%EC%82%AC%EC%9A%A9%ED%95%98%EC%9E%90-XGBoost


### 시험환경 체험
- 작업1형 : https://dataq.goorm.io/exam/116674/%EC%B2%B4%ED%97%98%ED%95%98%EA%B8%B0/quiz/1
- 작업2형 : https://dataq.goorm.io/exam/116674/%EC%B2%B4%ED%97%98%ED%95%98%EA%B8%B0/quiz/2
- 작업3형 : https://dataq.goorm.io/exam/116674/%EC%B2%B4%ED%97%98%ED%95%98%EA%B8%B0/quiz/3
- 빅분기 실기 문제 풀이 연습 사이트 : https://www.datamanim.com/dataset/03_dataq/re.html
https://colab.research.google.com/drive/1z3EpmaRWh4EVQ8LWG5INF1KC8vxpU2PG
- 빅분기 실기 준비 사이트(기출문제 풀이) : https://www.kaggle.com/datasets/agileteam/bigdatacertificationkr


### ML 주의사항
- submission 데이터 개수 X_test.csv 에 있는 데이터(행) 개수와 동일해야 함
- 전처리에서 '함정' 처리를 해야함
  - dtype : ML(numeric만 학습가능)은 object 학습 불가
  - invalid 문자포함 - 제거 (pandas 01.youtube - replace(regex=True)
  - Encoding : LabelEncoding, OneHotEncoding 
  - dfX -> X_train.csv, X_test.csv 같이 전처리 하는 것!
  - 범주 불일치에 대해서도 대응이 됨
  - 날짜/시간 : year, month, day, hour, weekday (minute, second -> 가짓수 많음)
- 시간제한 (1분)
  - SVM, Ensemble(estimators 개수 크게) - 시간 1분 꼭 확인
  - 최적의 parameter를 찾기위해 GridSearchCV를 사용했다면 - 주석취함 (찾은 parameter만 적어둠)
- 평가 (2항분류, 다항분류, 회귀)
  - r2_score(y_true, y_pred)
  - 분류 : precision, f1, recall : 다항(average={None, 'micro', 'macro', 'weighted'})
  - 분류 : roc_auc_score(y_true, y_proba) : 2항(1개 확률값), 다항(전체 확률값), average(위와 동일), multi_class='ovo'/'ovr'
  - 회귀 : msle(rmsle) -> 예측값에 음수가 있으면 안됨!
  - 평가 보정 : 예측값이 음수가 나올 수 없는 값(음수가 있는 확인하고, 0이나, 절대값 등으로 보정)


### 1.범주 불일치
- X를 합쳐서 전처리

In [ ]:
import pandas as pd
X_use = pd.DataFrame({'F1': ['A', 'B', 'O', 'A', 'B', 'O', 'A', 'B', 'O']})
F1 = X_use['F1'].astype('category').cat.codes  # A->0, B->1, O->2
print(F1.to_frame().T)

   0  1  2  3  4  5  6  7  8
0  0  2  1  0  2  3  1  2  3


In [ ]:
X_submission = pd.DataFrame({'F1': ['A', 'B', 'AB', 'A', 'B', 'O', 'AB', 'B', 'O']})
F1 = X_submission['F1'].astype('category').cat.codes  # A->0, B->1, O->2
print(F1.to_frame().T) # A->0  AB-> 1, B->2, O->3

   0  1  2  3  4  5  6  7  8
0  0  2  1  0  2  3  1  2  3


In [ ]:
dfX = pd.concat([X_use, X_submission])
F1 = dfX['F1'].astype('category').cat.codes
print(F1.to_frame().T)

   0  1  2  3  4  5  6  7  8  0  1  2  3  4  5  6  7  8
0  0  2  3  0  2  3  0  2  3  0  2  1  0  2  3  1  2  3


### 2.평가 함수 정리

#### 2-1.2항 분류

In [ ]:
# 2항 분류 
from sklearn.metrics import roc_auc_score, f1_score

def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)  # accuracy
    B = model.score(x_test, y_test)    # accuracy
    y_pred = model.predict(x_test)
    C = f1_score(y_test, y_pred)    
    y_proba = model.predict_proba(x_test)[:, 1]
    D = roc_auc_score(y_test, y_proba)
    return f'ACC: {A:.4f} {B:.4f}  F1: {C:.4f} AUC: {D:.4f}'

#### 2-2.다항 분류

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, roc_auc_score

def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    # f1_score, precision_score, recall_score 
    y_pred = model.predict(x_test)    
    C = f1_score(y_test, y_pred, average='macro')
    y_proba = model.predict_proba(x_test)
    D = roc_auc_score (y_test, y_proba, average='macro', multi_class='ovo')
    return f'ACC: {A:.4f} {B:.4f} F1: {C:.4f} AUC: {D:.4f}'


#### 2-3.회귀 

In [ ]:
import sklearn
print(help(sklearn))   #  package가 생각나지 않을 때

In [ ]:
# 철자 생각이 잘 안난다면!!
import sklearn.metrics
import numpy as np
print(np.array(dir(sklearn.metrics))[50:])

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_log_error as msle

# msle, rmsle -> 음수가 있어서는 안됨
# rmse, rmsle -> sklearn.metrics에 없음 
# -> mse ** 0.5 -> rmse, msle ** 0.5 -> rmsle
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)  # r2_score
    B = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    #y_pred[0] = -1
    y_pred = np.where(y_pred > 0, y_pred, -y_pred)  # 음수처리 방법
    C = mse(y_test, y_pred) ** 0.5
    D = mape(y_test, y_pred)
    E = msle(y_test, y_pred) ** 0.5  # 음수가 있어서는 안됨
    return f'R2 : {A:.4f} {B:.4f} rmse : {C:.4f}, mape: {D:.4f}, rmsle {E:.4f}'

# ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

### 3.help, dir 사용
- dir : 객체/모듈의 자주 사용되는 함수/클래스/값의 목록을 보여줌
- help : 모듈/함수(=메서드)의 parameter설명, 사용 예

In [ ]:
import pandas as pd
import sklearn
import sklearn.linear_model
from sklearn.linear_model import LogisticRegression

#print(help(sklearn))              # sklearn의 패키지 철자가 생각 나지 않을 때
#print(dir(sklearn.linear_model))  # 패키지 내부의 객체가 생각 나지 않을 때
#print(help(LogisticRegression))   # 객체 사용법이 생각 나지 않을 때 - 하단의 example

# pandas의 Series 객체의 메서드 사용법이 생각 나지 않을 때
#print(help(pd.Series.replace))

#print(type(pd.DataFrame.shape))  # property 라고 표시되면 값! -> help 없음
#print(type(pd.Series.replace))   # function 이라고 표시되면 함수! -> help 있음

### 3. 제2유형 접근방법
- 최선이 아닌 기본을 하자

빅분기 실기 예시/기출문제 
- 예시문제 LAB : https://colab.research.google.com/drive/1qzoKHby36udD7SuI12xcnVTqlbpAmFMP
- 예시문제 SOL : https://colab.research.google.com/drive/1GT14TseM2y5y7hzoDRQEIdzUuU1wwWzg
- 2회기출 LAB : https://colab.research.google.com/drive/1jSTnSmQeU8YQx3j2ZnSBkYiprWISZo6u 
- 2회기출 SOL : https://colab.research.google.com/drive/1UDlnnEIk55CRTggb8fKhTnalBS2qlkqk 
- 3회기출 LAB : https://colab.research.google.com/drive/1OMoKjt2noLrDtRDUUI56EJy66S_1Bo0B 
- 3회기출 SOL : https://colab.research.google.com/drive/14TRVkbfb8UjMs5OLU79ECZlIUTu6wuGj 
- 4회기출 LAB : https://colab.research.google.com/drive/1FGzIskDZoeCHbclZaHYvT3DCA78oHBIZ 
- 4회기출 SOL : https://colab.research.google.com/drive/1uwkE0ENjccekcSakxikkFPZSdlBiGAFK 
- 5회기출 2유형 LAB : https://colab.research.google.com/drive/1nW1ofFYysHGQ3Xczx225PqgiApnpAlwR
- 5회기출 2유형 SOL : https://colab.research.google.com/drive/1lwmlOf9q_8YTnqTUj1fSKY_qxLAKJOcg


## 3.기출문제 (빅분기 실기 2회)
- 고객 구매 데이터를 사용해서 고객이 주문한 물품이 제 시간에 도착여부(Reached.on.Time_Y.N) 예측
- X_train.csv, y_train.csv : 학습 및 학습평가에 사용하는 파일
- X_test.csv : 제출용 데이터 생성에 사용하는 파일
- roc_score를 사용하여 평가
- 아래의 형식으로 submission수험번호.csv 로 저장
- Reached.on.Time_Y.N은 제 시간에 도착할 확률
        ID,Reached.on.Time_Y.N
        7007,0.27419354838709675
        9793,0.4883720930232558
        6593,0.48174157303370785
        10527,0.3657243816254417

In [ ]:
# [0] x_train, x_test 데이터로 생성함
# 문제 출제 형식에 맞도록 변환한 것입니다.  시험에서는 이런 코드가 없습니다.
from sklearn.model_selection import train_test_split
import pandas as pd
dftot = pd.read_csv('https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/1st_Train.csv')
x_train, x_test = train_test_split(dftot, test_size=0.4, stratify=dftot['Reached.on.Time_Y.N'], random_state=0)
y_train = x_train[['Reached.on.Time_Y.N']]
x_train = x_train.drop(columns='Reached.on.Time_Y.N')
y_test = x_test[['Reached.on.Time_Y.N']]   # 이것은 시험에서는 제공되지 않음
x_test = x_test.drop(columns='Reached.on.Time_Y.N')

x_train.to_csv('x_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
x_test.to_csv('x_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False) # 이것은 시험에서는 제공되지 않음

In [ ]:
# [0] 사용 라이브러리 import
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# 2항 분류 
from sklearn.metrics import roc_auc_score, f1_score

def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)  # accuracy
    B = model.score(x_test, y_test)    # accuracy
    y_pred = model.predict(x_test)
    C = f1_score(y_test, y_pred)    
    y_proba = model.predict_proba(x_test)[:, 1]
    D = roc_auc_score(y_test, y_proba)
    return f'ACC: {A:.4f} {B:.4f}  F1: {C:.4f} AUC: {D:.4f}'


# 데이타가 많은경우 모두 출력 안되고 ... 으로 생략해서 출력됨.
pd.set_option('display.max_rows',500)    #출력할 max row를 지정
pd.set_option('display.max_columns',20)  #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format', '{:.4f}'.format)

# [1] x_train.csv, y_train.csv, x_test.csv 읽어오기
X = pd.read_csv('x_train.csv')
Y = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')
#print(X.shape, Y.shape, X_submission.shape)   # 제출! (6599, 11) (4400, 1)

# [2] X, X_submission 합치기 (필수는 아님)
# # 행을 삭제하지 말것, 순서변경하지 말 것
# - 다시 분리된 후에는 X_train.csv의 데이터는 행 삭제, 위치이동 편하게 하세요!
dfX = pd.concat([X, X_submission])
#dfX.info()
#print(dfX.describe())
# 범주형 : 'Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender' (Encoding 필요)
#           'Customer_care_calls', 'Customer_rating', 'Prior_purchases'
# 연속형 :  'Cost_of_the_Product', 'Weight_in_gms', 'Discount_offered'

dfXF = pd.get_dummies(dfX)
dfXF = dfXF.drop(columns=['ID', 'Gender_F']) # 변수 선택 (사용하지 않을 변수 제거)

# [3] dfX의 각 컬럼의 값 종류 개수 - 1인 것 제거하기 위해
#print(dfXF.nunique())

# [4] 스케일링 작업
#dfXF = pd.DataFrame(MinMaxScaler().fit_transform(dfXF))

# [5] X_use, X_submission 분리 
X_use = dfXF.iloc[:6599, :]
X_submission = dfXF.iloc[6599:, :]
Y_use = Y['Reached.on.Time_Y.N']
#print(X_use.shape, Y.shape, X_submission.shape)
#X_use.info()

# [6] train, test 분리
temp = train_test_split(X_use, Y_use, test_size = 0.2, 
                        stratify=Y_use, random_state=1234)
x_train, x_test, y_train, y_test = temp
#print([x.shape for x in [x_train, x_test, y_train, y_test]])
#print(Y_use.value_counts(normalize=True)) # 약간 불균형 (0.4 : 0.6)

# [7] 모델링, 평가
model1 = DecisionTreeClassifier(max_depth=3, random_state=1234)
model1.fit(x_train, y_train)
#print(model1.get_depth())
print(get_scores(model1, x_train, x_test, y_train, y_test))
#ACC: 0.6768 0.6667  F1: 0.6910 AUC: 0.7357 (3)

#model2 = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1234) 
#model2.fit(x_train, y_train)
#print(get_scores(model2, x_train, x_test, y_train, y_test))

#model3 = KNeighborsClassifier(12) 
#model3.fit(x_train, y_train)
#print(get_scores(model3, x_train, x_test, y_train, y_test))

# [8] 제출준비 및 제출
submission = pd.DataFrame()
submission['NY'] = model1.predict_proba(X_submission)[:, 1]
submission.to_csv('수험번호.csv', index=False)

# [9] 제출파일 확인
#temp = pd.read_csv('수험번호.csv')
#print(temp.shape)
#print(temp.head())

ACC: 0.6768 0.6667  F1: 0.6910 AUC: 0.7357


In [ ]:
y_true = pd.read_csv('y_test.csv')
y_proba = pd.read_csv('수험번호.csv')

D = roc_auc_score(y_true, y_proba)
print(round(D, 2))

0.74


### 4.기출문제 5회
- 벤츠 차량 가격 예측 : https://www.kaggle.com/datasets/mysarahmadbhat/mercedes-used-car-listing 
- train = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv 
- test = https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv
- 예측 변수 price, test.csv에 대해 price 값을 예측하여 제출, 제출 데이터 컬럼은 price만 존재해야함.
- 평가지표는 rmse

In [ ]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv')
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
# [0] import 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# msle, rmsle -> 음수가 있어서는 안됨
# rmse, rmsle -> sklearn.metrics에 없음 
# -> mse ** 0.5 -> rmse, msle ** 0.5 -> rmsle
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    y_pred = np.where(y_pred > 0, y_pred, -y_pred)  # 음수처리 방법
    C = mse(y_test, y_pred) ** 0.5
    D = mape(y_test, y_pred)
    E = msle(y_test, y_pred) ** 0.5  # 음수가 있어서는 안됨
    return f'R2 : {A:.4f} {B:.4f} rmse : {C:.4f}, mape: {D:.4f}, rmsle {E:.4f}'
# ---------------------------------------------------
# [1] 파일 읽어오기
train = pd.read_csv('train.csv')  # 종속변수 : price, 9823
test = pd.read_csv('test.csv')    # 제출용도 : (3296, 1)

X = train.drop(columns=['price'])
Y_use = train['price']
X_submission = test[X.columns]
#print(X.shape, X_submission.shape)

# [2] 데이터 탐색 - object, 결측치
# object : model, transmission, fuelType
#train.info()
#test.info()
#print(train.describe())
#print(train['engineSize'].unique())
#train[train['engineSize'] == 0]
#train['transmission'].value_counts().sort_index().to_frame().T

# train에는 있고, test에는 없는 데이터 존재 컬럼숫자가 다름 -> reindex해줘야함
# OneHotEncoding 사용 (X, X_submission 별도로 처리)
#dum_x = pd.get_dummies(X)
#dum_test = pd.get_dummies(X_submission)
#dum_test = dum_test.reindex(columns = dum_x.columns, fill_value=0)
#print(dum_x.columns)
#print(dum_test.columns)

#temp = pd.DataFrame()
#cat_features = ['model', 'transmission', 'fuelType']
#temp['train'] = train[cat_features].nunique()
#temp['test'] = test[cat_features].nunique()
#temp.T

# [2] X, X_submission 합치기
dfX = pd.concat([X, X_submission])
#dfX.info()

# [3] Encoding
#dfX['model'] = dfX['model'].astype('category').cat.codes
#dfX['transmission'] = dfX['transmission'].astype('category').cat.codes
#dfX['fuelType'] = dfX['fuelType'].astype('category').cat.codes

dfX = pd.get_dummies(dfX)

# [4] 스케일링
#dfX = pd.DataFrame(MinMaxScaler().fit_transform(dfX))

# [5] X, X_submission 분리
X_use = dfX.iloc[:9823, :]
X_submission = dfX.iloc[9823:, :]
#print(X_use.shape, Y_use.shape, X_submission.shape)
# print(X_use.info(), X_submission.info())

# [6] train, test 분리 ()
temp = train_test_split(X_use, Y_use, test_size=0.3, random_state=123)
x_train, x_test, y_train, y_test = temp
#print([x.shape for x in [x_train, x_test, y_train, y_test]])

# [7] 모델링, 평가

#model1 = DecisionTreeRegressor(max_depth=11, random_state=4321).fit(x_train, y_train)
#print(model1.get_depth())
#print(get_scores(model1, x_train, x_test, y_train, y_test))

model2 = RandomForestRegressor(n_estimators=300, random_state=4321).fit(x_train, y_train)
#print(get_scores(model2, x_train, x_test, y_train, y_test))  # rmse : 2905.5282


# [8] 파일 저장 및 제출
submission = pd.DataFrame()
submission['price'] = model2.predict(X_submission)
submission.to_csv('수험번호.csv', index=False)

# [9] 확인
#temp = pd.read_csv('수험번호.csv')
#print(temp.shape)
#print(temp.head())

(9823, 9) (3296, 9)
R2 : 0.9935 0.9408 rmse : 2854.0073, mape: 0.0668, rmsle 0.0972
(3296, 1)
       price
0  4493.2667
1 61753.9467
2 17482.3133
3 14844.3400
4 17001.9967


In [ ]:
test.nunique()

ID              3296
model             22
year              21
transmission       3
mileage         2788
fuelType           3
tax               29
mpg               94
engineSize        21
dtype: int64

In [ ]:
test['transmission'].value_counts().sort_index().to_frame().T

,Automatic,Manual,Semi-Auto
transmission,1209,385,1702


In [ ]:
# [0] import 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_log_error as msle

# msle, rmsle -> 음수가 있어서는 안됨
# rmse, rmsle -> sklearn.metrics에 없음 
# -> mse ** 0.5 -> rmse, msle ** 0.5 -> rmsle
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    #y_pred[0] = -1
    y_pred = np.where(y_pred > 0, y_pred, -y_pred)  # 음수처리 방법
    C = mse(y_test, y_pred) ** 0.5
    D = mape(y_test, y_pred)
    E = msle(y_test, y_pred) ** 0.5  # 음수가 있어서는 안됨
    return f'{A:.4f} {B:.4f} rmse : {C:.4f}, mape: {D:.4f}, rmsle {E:.4f}'

# ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.


# [1] 파일 불러오기
X_use = pd.read_csv('x_train.csv')
Y_use = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')

dfX = pd.concat([X_use, X_submission])

dfX2 = pd.get_dummies(dfX)

Y = Y_use['price']

# [4] X_useF, X_submissionF 로 분리
X_useF = dfX2.iloc[:len(X_use)]
X_submissionF = dfX2.iloc[len(X_use):]

# [5] train_test_split 
# stratify 는 분류모델에서 사용!  분류모델에서도 범주별 데이터가 2개 이상이어야 함
temp = train_test_split(X_useF, Y, test_size=0.25, random_state=0)
x_train, x_test, y_train, y_test = temp
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) 
# (7367, 5) (2456, 5) (7367,) (2456,)

model4 = RandomForestRegressor(random_state=1234)
model4.fit(x_train, y_train)
print(get_scores(model4, x_train, x_test, y_train, y_test))

submission = pd.DataFrame()
submission['price'] = model4.predict(X_submissionF)
submission.to_csv('수험번호.csv', index=False)

# temp = pd.read_csv('수험번호.csv')
# print(temp.shape)
# print(temp.head())